<a href="https://colab.research.google.com/github/thiagoscerqueira/fiap_techchallenge_03/blob/main/preprocess_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pré-processamento do dataset**

Aqui estamos efetuando a leitura do dataset "Amazon titles 1.3MM", arquivo `trn.json`, efetuando a limpeza dos dados e gerando um arquivo csv para posterior fine-tuning.



# **Instalando as bibliotecas necessárias**

In [ ]:
!pip install datasets
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset
import pandas as pd

**Aqui estamos carregando o dataset da amazon que foi armazenado em um dataset no Hugging Faces Hub**

In [ ]:
dataset = load_dataset('thiagoscerqueira/amazontitlestrn')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


trn.json:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2248619 [00:00<?, ? examples/s]

# **Tratamento dos dados**

Aqui estamos aplicando uma função de filtro e transformação em cada entrada do dataset.
Na transformação, consideramos apenas os campos "title" and "content".
Caso exista uma entrada sem um título ou descrição, esta entrada é desconsiderada.

In [ ]:
def filter_and_transform(example):
    title = example['title'].strip()
    content = example['content'].strip()
    if title and content:  # Check if both fields are non-empty
        return {
            'title': title,
            'content': content
        }
    return None  # Skip rows with empty 'title' or 'content'

In [ ]:
transformed_dataset = dataset['train'].filter(lambda example: filter_and_transform(example) is not None)

In [ ]:
print(transformed_dataset)

Dataset({
    features: ['title', 'content'],
    num_rows: 1390403
})


In [ ]:
transformed_dataset = transformed_dataset.map(filter_and_transform, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/1390403 [00:00<?, ? examples/s]

In [ ]:
print(transformed_dataset)

Dataset({
    features: ['title', 'content'],
    num_rows: 1390403
})


**Aqui estamos carregando o dataset tratado em um dataframe do pandas, facilitando a operação de geração de um arquivo csv a partir do dataset**

In [ ]:
df = pd.DataFrame(transformed_dataset)

**Neste trecho, estamos gerando o csv pelo dataframe do pandas e posteriormente salvando o arquivo gerado no Google Drive e fazendo upload do mesmo para o Hugging Face Hub.**

In [ ]:
from google.colab import drive
from huggingface_hub import HfApi, Repository
import shutil


drive.mount('/content/drive')


csv_file_path = '/content/drive/MyDrive/datasets/transformed_dataset.csv'

df.to_csv(csv_file_path, index=False, columns=['title', 'content'])

print(f"Dataset salvo na pasta {csv_file_path}.")

api = HfApi()
api.login()

repo_id = 'thiagoscerqueira/amazontitlestrn'
repo = Repository(local_dir='local-dataset', clone_from=repo_id)

shutil.copy(csv_file_path, 'local-dataset/transformed_dataset.csv')

repo.git_add('transformed_dataset.csv')
repo.git_commit('Add transformed_dataset.csv')
repo.git_push()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset successfully transformed and saved to /content/drive/MyDrive/datasets/transformed_dataset.csv.
